# llama_3.1 8B Instruct with Logits

This model could ben downloaded and deploy locally, despite its slow runtime. 

# Extract Logits

In [1]:
from transformers import LlamaForCausalLM, AutoTokenizer
import torch
from huggingface_hub import snapshot_download, login
import numpy as np
import os
from sentence_transformers import SentenceTransformer
from sklearn.cluster import AgglomerativeClustering
from collections import Counter

/Users/ivan/Documents/uni-projects/mlp-confidence-calibration-research/venv/lib/python3.13/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
login(token=os.environ["HF_TOKEN"])

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
# Load tokenizer and model
local_dir = snapshot_download(repo_id="meta-llama/Llama-3.2-3B-Instruct")
model = LlamaForCausalLM.from_pretrained(local_dir, torch_dtype=torch.float16, low_cpu_mem_usage=True)
tokeniser = AutoTokenizer.from_pretrained(local_dir)

Loading checkpoint shards: 100%|██████████| 2/2 [00:15<00:00,  7.68s/it]


In [4]:
def response_tokeniser_with_sgc(message_lst: list[dict], tokeniser: AutoTokenizer, model: LlamaForCausalLM, max_new_tokens = 5) -> tuple:
    flat_msg_list = []
    for x in range(len(message_lst)): flat_msg_list += [{"role":k , "content": v} for k, v in message_lst[x].items()]
    inputs = tokeniser(tokeniser.apply_chat_template(flat_msg_list, tokenize=False, add_generation_prompt=True), return_tensors="pt")
    with torch.no_grad():
        outputs = model.generate(
            inputs['input_ids'], 
            max_new_tokens=max_new_tokens, 
            return_dict_in_generate=True, 
            output_logits=True,
            output_scores=True,
            top_p =0.9
        )
    return "".join(tokeniser.batch_decode(outputs.sequences[:, inputs["input_ids"].shape[1]:], skip_special_tokens=True)), float(np.exp(model.compute_transition_scores(outputs.sequences, outputs.scores, normalize_logits=True).numpy(force=True)).mean())

In [9]:
response_tokeniser_with_sgc([{"user": "Who are the top riches people?"}], tokeniser, model, 256)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


("Here's a list of the top 10 richest people in the world, based on the Forbes Billionaires List 2023:\n\n1. **Bernard Arnault - Net Worth: $230 billion**\n   - Nationality: French\n   - Occupation: Business magnate, Investor\n   - Companies: LVMH (Moët Hennessy Louis Vuitton), Christian Dior, and others\n\n2. **Elon Musk - Net Worth: $190 billion**\n   - Nationality: South African-American\n   - Occupation: Entrepreneur, Business magnate\n   - Companies: SpaceX, Tesla, Neuralink, and others\n\n3. **Jeff Bezos - Net Worth: $140 billion**\n   - Nationality: American\n   - Occupation: Entrepreneur, Business magnate\n   - Companies: Amazon, Blue Origin\n\n4. **Bill Gates - Net Worth: $130 billion**\n   - Nationality: American\n   - Occupation: Business magnate, Philanthropist\n   - Companies: Microsoft\n\n5. **Warren Buffett - Net Worth: $120 billion**\n   - Nationality: American\n   - Occupation: Investor, Business magnate\n   - Companies: Berkshire Hathaway\n\n6. **Mark Zuckerberg - Net

In [ ]:
# msgs = [{"user": "What is the most populated city in Australia?"}, {"system": "provide the answer as concisely as possible"}]
# multi_response = [response_tokeniser_with_sgc(msgs, tokeniser, model) for _ in range(50)]

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end gene

In [ ]:
# multi_response = list(zip(np.random.default_rng().choice(["A", "B", "C", "D"], 50, p=[0.7, 0.1, 0.1, 0.1]).tolist(), (np.random.uniform(0, 1, 50)).tolist()))
# multi_response

[('A', 0.5671795555608957),
 ('B', 0.006688037935689839),
 ('A', 0.32312966452742575),
 ('B', 0.8913660505211838),
 ('D', 0.023431141726125593),
 ('A', 0.3762705809561574),
 ('B', 0.7751518757390848),
 ('A', 0.29307948853273835),
 ('A', 0.44219249879984657),
 ('C', 0.6566782698658681),
 ('A', 0.6275072407590258),
 ('D', 0.17891707302661397),
 ('A', 0.7386071668341867),
 ('A', 0.4469239622093574),
 ('A', 0.7052241339770617),
 ('D', 0.12138475609063626),
 ('A', 0.2648679930555855),
 ('A', 0.7708806776946763),
 ('A', 0.40559243009857493),
 ('A', 0.3923507461217316),
 ('A', 0.5676950431328948),
 ('A', 0.9653199999268453),
 ('C', 0.9459039770411337),
 ('C', 0.8934748058898486),
 ('A', 0.8209174345364296),
 ('A', 0.4195759173233674),
 ('A', 0.6251506958176762),
 ('A', 0.2397868251522951),
 ('B', 0.7699579218427463),
 ('A', 0.6657001051039136),
 ('A', 0.9231818101195042),
 ('A', 0.9335733020135215),
 ('C', 0.2195753993282189),
 ('A', 0.12710028826051967),
 ('A', 0.9892971917144789),
 ('C', 0.

In [ ]:
def get_semantic_clusters(multi_response):
    lnll_lst = [(x)[1] for x in multi_response]
    response_list = [x[0] for x in multi_response]
    distance_threshold = 0.3
    model_name="all-MiniLM-L6-v2"
    embeddings = SentenceTransformer(model_name).encode(response_list)
    clustering = AgglomerativeClustering(n_clusters=None, distance_threshold=distance_threshold, metric="cosine", linkage="average")
    labels = clustering.fit_predict(embeddings)
    return response_list, lnll_lst, labels

def get_mcq_clusters(multi_response):
    lnll_lst = [(x)[1] for x in multi_response]
    response_list = [x[0] for x in multi_response]
    choice_map = dict().setdefault(0)
    choice_map = {'A': 1, 'B': 2, 'C': 3, 'D': 4, 'a': 1, 'b': 2, 'c': 3, 'd': 4}
    labels = [choice_map[c] for c in response_list]
    return response_list, lnll_lst, labels

response_list, lnll_lst, labels = get_mcq_clusters(multi_response)

# 1. Single Generation Confidence (SGC)

In [ ]:
def single_generation_confidence(lnll_lst, response_list, labels):
    return response_list[0], lnll_lst[0]

In [ ]:
single_generation_confidence(lnll_lst, response_list, labels)

('A', 0.5671795555608957)

# 2. Empirical Semantic Confidence (E-SC)

From the $m$ responses grouped into $k$ clusters, we select the final output by identifying the most confident cluster $C^*$ and choosing the response within it with the highest $\text{LN-LL: } y^*= \text{argmax}_{y∈C^*}\bar{l}(y |x).$

In [ ]:
def empirical_semantic_confidence(lnll_lst, response_list, labels):
    counts = Counter(labels)
    opt_cluster, opt_conf = max([(int(cluster_id), count/sum(counts.values())) for cluster_id, count in counts.items()], key=lambda x: x[1])
    optimal_response = max([(response_list[i], lnll_lst[i]) for i, label in enumerate(labels) if label == opt_cluster], key=lambda x: x[1])[0]
    return optimal_response, opt_conf

In [ ]:
empirical_semantic_confidence(lnll_lst, response_list, labels)

('A', 0.72)

# 3. Likelihood-based Semantic Confidence (L-SC)

In [ ]:
def likelihood_based_semantic_confidence(lnll_lst, response_list, labels):
    clustered = []
    # for each cluster calculat the following:
    for c in np.unique(labels):
        sum_ci = sum([lnll_lst[i] for i in range(len(lnll_lst)) if labels[i] == c]) # s(Ci | x) = sum(LN-LL)
        clustered.append((int(c), sum_ci))
    total_lsc = sum([x[1] for x in clustered])
    lsc = [(c[0], c[1] / total_lsc) for c in clustered]
    
    opt_cluster, opt_conf = max(lsc, key=lambda x: x[1])
    optimal_response = max([(response_list[i], lnll_lst[i]) for i, label in enumerate(labels) if label == opt_cluster], key=lambda x: x[1])[0]

    return optimal_response, opt_conf

In [ ]:
likelihood_based_semantic_confidence(lnll_lst, response_list, labels)

('A', 0.7198496955065203)

# 4. Mean Likelihood-based Semantic Confidence (ML-SC)

In [ ]:
def mean_likelihood_based_semantic_confidence(lnll_lst, response_list, labels):
    clustered = []
    # for each cluster calculate s_bar(Ci | x):
    for c in np.unique(labels):
        sum_ci = sum([lnll_lst[i] for i in range(len(lnll_lst)) if labels[i] == c]) # s_bar(Ci | x) = sum(LN-LL) in cluster Ci
        clustered.append((int(c), sum_ci / Counter(labels)[c])) # s_bar(Ci | x) = sum(LN-LL) / count(Ci)
    total_mlsc = sum([x[1] for x in clustered])
    mlsc = [(c[0], c[1] / total_mlsc) for c in clustered]
    
    opt_cluster, opt_conf = max(mlsc, key=lambda x: x[1])
    optimal_response = max([(response_list[i], lnll_lst[i]) for i, label in enumerate(labels) if label == opt_cluster], key=lambda x: x[1])[0]

    return optimal_response, opt_conf

In [ ]:
mean_likelihood_based_semantic_confidence(lnll_lst, response_list, labels)

('C', 0.3746192061428716)

# 5. Bayesian Semantic Confidence (B-SC)

In [ ]:
def bayesian_semantic_confidence(lnll_lst, response_list, labels):
    clustered = []
    for c in np.unique(labels):
        pi = Counter(labels)[c] / len(labels) # pi = count(Ci) / count
        joint_lnll = np.prod([lnll_lst[i] for i in range(len(lnll_lst)) if labels[i] == c]) # joint lnll = ∏(LN-LL) in cluster Ci
        clustered.append((int(c), joint_lnll * pi)) # joint_lnll * pi
    
    total_bsc = sum([x[1] for x in clustered])
    bsc = [(c[0], float(c[1] / total_bsc)) for c in clustered]

    opt_cluster, opt_conf = max(bsc, key=lambda x: x[1])
    optimal_response = max([(response_list[i], lnll_lst[i]) for i, label in enumerate(labels) if label == opt_cluster], key=lambda x: x[1])[0]

    return optimal_response, opt_conf

In [ ]:
bayesian_semantic_confidence(lnll_lst, response_list, labels)

('C', 0.987692041934822)